In [8]:
import os
import errno
import numpy as np
import deepcell
from deepcell.training import train_model_retinanet

In [2]:
# Download the data (saves to ~/.keras/datasets)
filename = 'mousebrain.npz'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.mousebrain.load_data(filename)

In [3]:
print('X.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

X.shape: (176, 15, 256, 256, 1)
y.shape: (176, 15, 256, 256, 1)


In [4]:
# the path to the data file is currently required for `train_model_()` functions

# NOTE: Change DATA_DIR if you are not using `deepcell.datasets`
DATA_DIR = os.path.expanduser(os.path.join('~', '.keras', 'datasets'))

DATA_FILE = os.path.join(DATA_DIR, filename)

# confirm the data file is available
assert os.path.isfile(DATA_FILE)

In [5]:
# Set up other required filepaths

# If the data file is in a subdirectory, mirror it in MODEL_DIR and LOG_DIR
PREFIX = os.path.relpath(os.path.dirname(DATA_FILE), DATA_DIR)

ROOT_DIR = '/data'  # TODO: Change this! Usually a mounted volume
MODEL_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'models', PREFIX))
LOG_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'logs', PREFIX))

# create directories if they do not exist
for d in (MODEL_DIR, LOG_DIR):
    try:
        os.makedirs(d)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

In [6]:
from tensorflow.keras.optimizers import SGD, Adam
from deepcell.utils.train_utils import rate_scheduler

model_name = 'mrcnn_model_3D'
backbone = 'resnet50_3d'  # vgg16, vgg19, resnet50, densenet121, densenet169, densenet201

n_epoch = 3  # Number of training epochs
test_size = .10  # % of data saved as test
lr = 1e-5

optimizer = Adam(lr=lr, clipnorm=0.001)

lr_sched = rate_scheduler(lr=lr, decay=0.99)

batch_size = 8

num_classes = 1  # "object" is the only class

In [ ]:
from deepcell import model_zoo
print(backbone)
model = model_zoo.MaskRCNN_3D(
    backbone=backbone,
    input_shape=(120, 256, 256, 1),
    class_specific_filter=False,
    num_classes=num_classes)


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=None, loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)